In [ ]:
import numpy as np
import os, shutil
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image

def RetrainCNN():
    epochs = 60 #Number of epochs
    learningrate = 0.0005 #Learning rate for optimizer 
    mfolder = "./models1/" #Folder with saved models in h5 format
    afolder = "./sorted/predicta/ab/" #Image folder with images to test on model
    efolder = "./sorted/ce/e/" #Image folder with images to test on model
    keepoldmodels = True #Do you want to delete left over models that aren't being used? False=delete, True=Keep


    #File preliminaries before training
    if not os.path.exists('./models1'): #Create folder for models
        os.makedirs('./models1')
    zfilelist = os.listdir('./tempfiles/') #Make list of user image filenames
    for zfile in zfilelist: #Copy user abnormal and echolocation files for training
        img=Image.open("./tempfiles/"+zfile).convert('RGB')
        if '{}'.format(zfile[0:1]) == 'a':
            img.save("sorted/train/abnormal/"+zfile)
        if '{}'.format(zfile[0:1]) == 'e':
            img.save("sorted/train/echolocation/"+zfile)
        
    #Tensorboard functions
    checkpointvacc = ModelCheckpoint('models1/{epoch:02d}.h5', monitor='val_acc', mode='max',
                                 save_best_only=True, verbose=0)
    checkpointvloss = ModelCheckpoint('models1/{epoch:02d}.h5', monitor='val_loss', mode='min',
                                 save_best_only=True, verbose=0)

    #Setup to stream images for test and training. Image augmentation can be added below
    train_data = ImageDataGenerator(rescale=1./255)
    test_data = ImageDataGenerator(rescale=1./255)
    train_generator = train_data.flow_from_directory('sorted/train',batch_size=32,class_mode='binary', target_size=(200,300))
    test_generator = test_data.flow_from_directory('sorted/test',batch_size=32,class_mode='binary',target_size=(200,300))

    #Create model layers
    model = Sequential()
    model.add(Conv2D(96, (5, 5), strides=5, input_shape=(200, 300, 3)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Conv2D(128, (5, 5), strides=5))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))      
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    #Set model parameters
    adam = optimizers.adam(lr=learningrate)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.fit_generator(
        train_generator,
        steps_per_epoch=80,
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=20,
        callbacks=[checkpointvloss,checkpointvacc])
    model.save('./models1/Modelatepoch'+str(epochs)+'.h5')
    
    #Evaluate models-------------------------------
    for mdl in range(20): #Remove unwanted models
        if os.path.exists(mfolder+str(mdl)+".h5"):
            os.remove(mfolder+str(mdl)+".h5")
        if os.path.exists(mfolder+"0"+str(mdl)+".h5"):
            os.remove(mfolder+"0"+str(mdl)+".h5")
    shutil.copy2('./CNN.h5', './models1') #Copy main model into model folder to be tested with others created    
    mfilelist = os.listdir(mfolder) #Make list of model filenames
    afilelist = os.listdir(afolder) #Make list of echo image filenames
    efilelist = os.listdir(efolder) #Make list of abnormal image filenames
    
    #Evaluate models against images in echolocation folder 
    echmodel = [0] * len(mfilelist)
    for mdl in range(len(mfilelist)):
        model = load_model(mfolder+mfilelist[mdl])
        inac=0
        for inum in range(len(efilelist)):#Go through images 1 by 1 and run predictions on model
            image = load_img(efolder + efilelist[inum], target_size=(200,300))
            imagearray = img_to_array(image) #Make image into an array
            imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
            imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
            prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
            if prediction[0]<0.9:
                inac += 1
        echmodel[mdl]=inac
        print('Model {0:s} missed count = {1:2d}'.format(mfilelist[mdl],echmodel[mdl]))
    for count in range(len(echmodel)): #Remove models with echolocation count >25
        if echmodel[count]>25: #Models with echolocation inaccuracy> 25 will not be evaluated in the abnormal predictions
            os.remove(mfolder+mfilelist[count])
            
    #Evaluate remaining models against images in abnormal folder 
    mfilelist = os.listdir(mfolder) #Make list of remaining model filenames
    print('mfile list after echo scan',mfilelist)
    abest=0 #Records score of model with best abnormal accuracy of the remaining models
    for mdl in range(len(mfilelist)):
        model = load_model(mfolder+mfilelist[mdl])
        abcount=0
        for inum in range(len(afilelist)):#Go through images 1 by 1 and run predictions on model
            image = load_img(afolder + afilelist[inum], target_size=(200,300))
            imagearray = img_to_array(image) #Make image into an array
            imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
            imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
            prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
            if prediction[0]<0.7:
                abcount += 1
        if abcount >= abest: #Determine which model has best abnormal file accuracy
            abest = abcount
            ambest = mdl
        print('Model {0:s} with abcount {1:2d}. Best model is {2:s}'.format(mfilelist[mdl],abcount,mfilelist[ambest]))
    
    #Copy new model and use if needed
    print('Best model is {0:s}'.format(mfilelist[ambest]))
    if mfilelist[ambest] != 'CNN.h5':
        os.remove("./CNN.h5") #Remove last used model
        shutil.copy2('./models1/'+mfilelist[ambest], './') #Copy new model 
        os.rename(mfilelist[ambest],'./CNNa.h5') #Rename new model to be used by system
        
    #Remove uploaded abnormal and echolocation files used for training
    for zfile in zfilelist: 
        if os.path.exists('sorted/train/abnormal/'+zfile):
            os.remove('sorted/train/abnormal/'+zfile)
        if os.path.exists('sorted/train/echolocation/'+zfile):
            os.remove('sorted/train/echolocation/'+zfile)    
        if os.path.exists('sorted/test/abnormal/'+zfile):
            os.remove('sorted/test/abnormal/'+zfile)
        if os.path.exists('sorted/test/echolocation/'+zfile):
            os.remove('sorted/test/echolocation/'+zfile)
            
    #Delete or keep all models in folder that were being evaluated
    if not keepoldmodels:
        for mfile in mfilelist:
            os.remove(mfolder+mfile)
RetrainCNN()